# Adding new fields of information to National Report Cards centroid service

In [1]:
import arcgis
from arcgis.gis import GIS, GroupManager
from arcgis.features import FeatureLayerCollection
from arcgis.features import FeatureLayer
from copy import deepcopy
from arcgis import geometry
import re
from pprint import pprint
import pandas as pd

In [2]:
env_path = ".env"
with open(env_path) as f:
    env = {}
    for line in f:
        env_key, _val = line.split("=")
        env_value = _val.split("\n")[0]
        env[env_key] = env_value

In [3]:
aol_password = env['aol_key']
aol_username = env['aol_username']

In [4]:
gis = GIS("https://eowilson.maps.arcgis.com", aol_username, aol_password)

In [5]:
def create_strict_reg_exp(to_search):
    try:
        reg_exp = f"^{to_search}$"
        #logging.info(f"regular expression is: {reg_exp}")
    except:
        print("There was a problem with the string.")
    return reg_exp

In [6]:
def findItemGetID(csvName, gis):
    try:
        searched_item = gis.content.search(csvName, item_type = "Feature Layer")
        for i in searched_item:
            reg_exp = create_strict_reg_exp(csvName)
            if re.search(reg_exp, i.title)!= None:    
                #logging.info(f"{csvName} has the id: {i.id}")
                return i.id
    except:
        print("There was a problem finding the item")

## Get item that has to be updated

In [9]:
item_id = findItemGetID("gadm_centroid", gis)

In [10]:
item = gis.content.get(item_id)

In [11]:
flayer = item.layers[0]

## Process the csv so it can have matching field with the service

In [13]:
n_by_taxa = pd.read_csv("/he-scratchfolder/data/Terrestrial_vertebrate_groups_by_country_20200617.csv")

In [14]:
n_by_taxa_wide = n_by_taxa.pivot_table(index=['iso3', 'countryname'], columns='speciesgroup', values='nspecies')

In [15]:
n_by_taxa_wide['nspecies'] = n_by_taxa_wide.sum(axis = 1)

In [16]:
n_by_taxa_wide.index.name = 'iso3'
n_by_taxa_wide.reset_index(inplace=True)

In [17]:
n_by_taxa_wide = n_by_taxa_wide.fillna(0)

In [18]:
n_by_taxa_wide.head()

speciesgroup,iso3,countryname,amphibians,birds,mammals,reptiles,nspecies
0,ABW,Aruba,3.0,92.0,4.0,32.0,131.0
1,AFG,Afghanistan,9.0,371.0,139.0,145.0,664.0
2,AGO,Angola,128.0,917.0,299.0,336.0,1680.0
3,AIA,Anguilla,2.0,106.0,5.0,12.0,125.0
4,ALA,Åland,5.0,112.0,11.0,4.0,132.0


In [19]:
n_by_taxa_wide.shape

(255, 7)

## Prepare the service to have more fields

In [32]:
#Get the existing list of fields on the published feature layer
flayer_fields = flayer.manager.properties.fields

In [33]:
for field in flayer_fields:
    print(f"{field.name:13}|  {field.alias:13}|  {field.type:25}|  {field.sqlType}")

OBJECTID_1   |  OBJECTID_1   |  esriFieldTypeOID         |  sqlTypeOther
GID_0        |  GID_0        |  esriFieldTypeString      |  sqlTypeOther
NAME_0       |  NAME_0       |  esriFieldTypeString      |  sqlTypeOther
jpg_url      |  jpg_url      |  esriFieldTypeString      |  sqlTypeOther
OBJECTID     |  OBJECTID     |  esriFieldTypeInteger     |  sqlTypeOther
GID          |  GID          |  esriFieldTypeString      |  sqlTypeOther
Area         |  Area         |  esriFieldTypeInteger     |  sqlTypeOther
GNI_PPP      |  GNI_PPP      |  esriFieldTypeDouble      |  sqlTypeOther
Protected    |  Protected    |  esriFieldTypeInteger     |  sqlTypeOther
HM_0         |  HM_0         |  esriFieldTypeInteger     |  sqlTypeOther
HM_low       |  HM_low       |  esriFieldTypeInteger     |  sqlTypeOther
HM_moderate  |  HM_moderate  |  esriFieldTypeInteger     |  sqlTypeOther
HM_high      |  HM_high      |  esriFieldTypeInteger     |  sqlTypeOther
Population2016|  SUM          |  esriFieldTypeDoubl

In [34]:
# get a template field with the type of interest in this case esriFieldTypeInteger
template_field = dict(deepcopy(flayer_fields[6]))
template_field

{'name': 'Area',
 'type': 'esriFieldTypeInteger',
 'alias': 'Area',
 'sqlType': 'sqlTypeOther',
 'nullable': True,
 'editable': True,
 'visible': True,
 'domain': None,
 'defaultValue': None}

In [140]:
# get the list of new fields to add from the third spreadsheet, that are not in spread sheets 1,2
# new_field_names = list(n_by_taxa_wide.columns.difference(sdf.columns))
# new_field_names
# when doing this there are two fields iso3 and country that are kept and they are not needed as the join of the tables is done feature by feature

['amphibians',
 'birds',
 'countryname',
 'iso3',
 'mammals',
 'nspecies',
 'reptiles']

In [35]:
new_field_names = ['amphibians', 'birds', 'mammals', 'nspecies', 'reptiles']
new_field_names

In [37]:
fields_to_be_added = []
for new_field_name in new_field_names:
    current_field = deepcopy(template_field)
    if new_field_name.lower() == 'class':
        current_field['sqlType'] = 'sqlTypeVarchar'
        current_field['type'] = 'esriFieldTypeInteger'
        #current_field['length'] = 8000
    current_field['name'] = new_field_name.lower()
    current_field['alias'] = new_field_name
    fields_to_be_added.append(current_field)
    
len(fields_to_be_added)

5

In [38]:
flayer.manager.add_to_definition({'fields':fields_to_be_added})

{'success': True}

In [39]:
new_fields = flayer.manager.properties.fields
len(new_fields)

31

In [40]:
for field in new_fields:
    print(f"{field.name:10}|  {field.type}")

OBJECTID_1|  esriFieldTypeOID
GID_0     |  esriFieldTypeString
NAME_0    |  esriFieldTypeString
jpg_url   |  esriFieldTypeString
OBJECTID  |  esriFieldTypeInteger
GID       |  esriFieldTypeString
Area      |  esriFieldTypeInteger
GNI_PPP   |  esriFieldTypeDouble
Protected |  esriFieldTypeInteger
HM_0      |  esriFieldTypeInteger
HM_low    |  esriFieldTypeInteger
HM_moderate|  esriFieldTypeInteger
HM_high   |  esriFieldTypeInteger
Population2016|  esriFieldTypeDouble
max_amph  |  esriFieldTypeInteger
max_bird  |  esriFieldTypeInteger
max_mamm  |  esriFieldTypeInteger
max_rept  |  esriFieldTypeInteger
max_cact  |  esriFieldTypeInteger
max_coni  |  esriFieldTypeInteger
max_all   |  esriFieldTypeInteger
sentence  |  esriFieldTypeString
prop_protected|  esriFieldTypeDouble
gadm_prop_COUNT|  esriFieldTypeInteger
N_SPECIES |  esriFieldTypeInteger
SPI       |  esriFieldTypeDouble
amphibians|  esriFieldTypeInteger
birds     |  esriFieldTypeInteger
mammals   |  esriFieldTypeInteger
nspecies  |  

## Update the data in the new fields with the data from the csv

In [41]:
fset2 = flayer.query()
features2 = fset2.features

In [44]:
features_for_update = []
for country_id in n_by_taxa_wide['iso3']:
    try:
        # get the matching row from csv
        matching_row = n_by_taxa_wide.where(n_by_taxa_wide.iso3 == country_id).dropna()

        print(str(country_id) + " Adding additional attributes for: " + matching_row['iso3'].values[0])

        # get the feature to be updated
        assert  len([f for f in features2 if f.attributes['GID_0'] == country_id]),  "country not matched"
        original_feature = [f for f in features2 if f.attributes['GID_0'] == country_id][0]
        feature_to_be_updated = deepcopy(original_feature)

        # assign the updated values
        feature_to_be_updated.attributes['amphibians'] = matching_row['amphibians'].values[0]
        feature_to_be_updated.attributes['birds'] = int(matching_row['birds'])
        #feature_to_be_updated.attributes['countryname'] = str(matching_row['countryname'])
        feature_to_be_updated.attributes['mammals'] = int(matching_row['mammals'])
        #feature_to_be_updated.attributes['iso3'] = str(matching_row['iso3'])
        feature_to_be_updated.attributes['nspecies'] = int(matching_row['nspecies'])
        feature_to_be_updated.attributes['reptiles'] = int(matching_row['reptiles'])

        #add this to the list of features to be updated
        features_for_update.append(feature_to_be_updated)
        #print(str(country_id) + " Done additional attributes for: " + matching_row['countryname'].values[0])
    except:
        print(f"{country_id} not available in service")

ABW Adding additional attributes for: ABW
AFG Adding additional attributes for: AFG
AGO Adding additional attributes for: AGO
AIA Adding additional attributes for: AIA
ALA Adding additional attributes for: ALA
ALB Adding additional attributes for: ALB
AND Adding additional attributes for: AND
ARE Adding additional attributes for: ARE
ARG Adding additional attributes for: ARG
ARM Adding additional attributes for: ARM
ASM Adding additional attributes for: ASM
ATA Adding additional attributes for: ATA
ATF Adding additional attributes for: ATF
ATG Adding additional attributes for: ATG
AUS Adding additional attributes for: AUS
AUT Adding additional attributes for: AUT
AZE Adding additional attributes for: AZE
BDI Adding additional attributes for: BDI
BEL Adding additional attributes for: BEL
BEN Adding additional attributes for: BEN
BES Adding additional attributes for: BES
BFA Adding additional attributes for: BFA
BGD Adding additional attributes for: BGD
BGR Adding additional attributes f

In [45]:
flayer.edit_features(updates= features_for_update)

{'addResults': [],
 'updateResults': [{'objectId': 1,
   'uniqueId': 1,
   'globalId': None,
   'success': True},
  {'objectId': 2, 'uniqueId': 2, 'globalId': None, 'success': True},
  {'objectId': 3, 'uniqueId': 3, 'globalId': None, 'success': True},
  {'objectId': 4, 'uniqueId': 4, 'globalId': None, 'success': True},
  {'objectId': 5, 'uniqueId': 5, 'globalId': None, 'success': True},
  {'objectId': 6, 'uniqueId': 6, 'globalId': None, 'success': True},
  {'objectId': 7, 'uniqueId': 7, 'globalId': None, 'success': True},
  {'objectId': 8, 'uniqueId': 8, 'globalId': None, 'success': True},
  {'objectId': 9, 'uniqueId': 9, 'globalId': None, 'success': True},
  {'objectId': 10, 'uniqueId': 10, 'globalId': None, 'success': True},
  {'objectId': 11, 'uniqueId': 11, 'globalId': None, 'success': True},
  {'objectId': 12, 'uniqueId': 12, 'globalId': None, 'success': True},
  {'objectId': 13, 'uniqueId': 13, 'globalId': None, 'success': True},
  {'objectId': 14, 'uniqueId': 14, 'globalId': Non